In [1]:
import numpy as np
import copy

## Find subnetworks

In [94]:
# Create a dictionary from adjaciency matrix
# key (each node) and value (the corresponding pre-nodes)

def nodes_dict(A): # A: adjaciency matrix
  B = {} # initalize an empty dictionary to store the nodes and edges
  for i in range(len(A)):
    quan_list = np.array(list(set(A[:,i]))) # at each column grab the values going from that node
    quan_list = quan_list[quan_list != 0]

    pre_nodes = []
    for j in quan_list:
      pre_nodes.append(list(np.where(A[:,i]==j)[0])) # add to the nodes list for each quantity
    B[i] = pre_nodes # save the nodes

  return B # return the dictionary

In [95]:
# Create a list of edge-lists of all subnetworks
# x0: starting node
# path: node flow -- nodes_dict(A)

def find_subnws(x0,path):
  D,Track = {},path.copy()
  NWs = []
  node_list, edge_list = [], []
  branch,restore = [],[]
  check = [x0]
  next = 1

  while(1):
    while(next):
      next,rr = [],[]
      for x0 in check:
        if x0 not in node_list:
          D[x0] = Track[x0][0]
        if len(Track[x0]) > 1:
          Track[x0] = Track[x0][1:]
          branch.append([node_list.copy(),edge_list.copy(),check.copy()])
          restore.append(check)
        node_list.append(x0)
        for i in D[x0]:
          if [x0,i] not in edge_list:
            edge_list.append([x0,i])
          if i not in node_list:
            next.append(i)
      check = next.copy()
    NWs.append(edge_list)
    if branch:
      node_list, edge_list, check = branch[-1]
      for pp in restore[-1]:
        for kk in path[pp]:
          for ll in kk:
            if ll not in node_list:
              Track[ll] = path[ll]
      branch.pop()
      restore.pop()
      next = 1
    else:
      break
  return NWs

In [96]:
# create edge-list from adjacency matrix
def adj_to_edge(A): # A is an m,n np.array
  m,n = np.shape(A)
  edges = []
  for r in range(m):
    for c in range(n):
      if A[r,c] != 0: edges.append([r,c,A[r,c]])
  return np.array(edges)

In [97]:
# create adjacency matrix from edge-list
def edge_to_adj(E,n): # E is an edge-list of an nxn matrix
  A = np.zeros((n,n))
  for r,c in E:
    A[c,r] = 1
  return A

## Test

In [98]:
# starting node
x0 = 3
# adjacency matrix
a = np.array([[0,1,0,0],[1,0,2,2],[3,0,0,0],[3,0,0,0]])
node_flow = nodes_dict(a)
NWs = find_subnws(x0,node_flow)

print(node_flow)
print(a,'\n\n')
print(f'Number of subnetworks: {len(NWs)}')
print(f'Starting node: {x0}')
for i in NWs:
  print(i)
  print(edge_to_adj(i,4),'\n')

{0: [[1], [2, 3]], 1: [[0]], 2: [[1]], 3: [[1]]}
[[0 1 0 0]
 [1 0 2 2]
 [3 0 0 0]
 [3 0 0 0]] 


Number of subnetworks: 2
Starting node: 3
[[3, 1], [1, 0], [0, 1]]
[[0. 1. 0. 0.]
 [1. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 

[[3, 1], [1, 0], [0, 2], [0, 3], [2, 1]]
[[0. 1. 0. 0.]
 [0. 0. 1. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]] 



In [99]:
transitions = []
x0 = 0
for i in range(len(node_flow[x0])):
    print('i', i)
    current = x0
    trans = []
    for node in node_flow[x0][i]:
        print('node', node)
        trans.append((current, node))
        for node2 in node_flow[node]:
            trans.append((node, node2[0]))
    transitions.append(trans)

transitions


i 0
node 1
i 1
node 2
node 3


[[(0, 1), (1, 0)], [(0, 2), (2, 1), (0, 3), (3, 1)]]

In [105]:
temp_dic = copy.deepcopy(node_flow)
node = 0
temp_dic
edges = []

In [107]:
new_nodes = temp_dic[node][0]
for n in new_nodes:
    edges.append((node, n))
    print(temp_dic[n])

[[0]]


In [108]:
edges

[(0, 1)]